In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delhi-5-minute-electricity-demand-for-forecasting/powerdemand_5min_2021_to_2024_with weather.csv


In [2]:
# Core
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Date handling
import datetime as dt

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer

# Model Selection
from sklearn.model_selection import train_test_split

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Machine Learning Models
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Utility
import warnings
warnings.filterwarnings('ignore')


2025-10-06 20:01:23.776171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759780884.154081      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759780884.262555      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import requests
import pickle
import datetime
import json
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb

In [4]:

electricity = pd.read_csv('/kaggle/input/delhi-5-minute-electricity-demand-for-forecasting/powerdemand_5min_2021_to_2024_with weather.csv')

COUNTRY = "IN"
YEAR = 2025
CITY = "Delhi"
lat, lon = 28.6139, 77.2090

In [5]:
#Holidays
if 'timestamp' not in electricity.columns:
    if 'datetime' in electricity.columns:
        electricity.rename(columns={'datetime': 'timestamp'}, inplace=True)
    elif electricity.index.name == 'timestamp' or isinstance(electricity.index, pd.DatetimeIndex):
        electricity = electricity.reset_index()  # move index to column
    else:
        raise KeyError("No 'timestamp' column or datetime index found in electricity DataFrame.")

# Ensure timestamp is datetime type
electricity['timestamp'] = pd.to_datetime(electricity['timestamp'])
def fetch_calendar_data(api_key="BKjmdqqX4JDnrVyeFUaigOCQdRdYJbck", country="IN", year=2025):
    url = f"https://calendarific.com/api/v2/holidays?&api_key={api_key}&country={country}&year={year}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        holidays = data['response']['holidays']
        if not holidays:
            return pd.DataFrame()
        df = pd.DataFrame([{
            "date": h['date']['iso'],
            "name": h.get('name', None),
            "type": h.get('type', [None])[0] if h.get('type') else None
        } for h in holidays])
        df['date'] = pd.to_datetime(df['date'].apply(lambda x: x.split("T")[0]), errors='coerce')
        return df
    else:
        print(f"Error fetching API data: {response.status_code}")
        return pd.DataFrame()

calendar_df = fetch_calendar_data()
print(calendar_df.head())

        date                       name              type
0 2025-01-01             New Year's Day  Optional holiday
1 2025-01-02       Last day of Hanukkah        Observance
2 2025-01-06  Guru Govind Singh Jayanti  Optional holiday
3 2025-01-13                      Lohri        Observance
4 2025-01-14                     Pongal          Hinduism


In [7]:
min_date = electricity['timestamp'].min().date()
max_date = electricity['timestamp'].max().date()
daily_calendar = pd.DataFrame({"date": pd.date_range(start=min_date, end=max_date, freq='D')})

calendar_df['date_only'] = calendar_df['date'].dt.date
daily_calendar['date_only'] = daily_calendar['date'].dt.date

daily_calendar = daily_calendar.merge(
    calendar_df[['date_only', 'name', 'type']],
    on='date_only',
    how='left'
)

daily_calendar['name'] = daily_calendar['name'].fillna('None')
daily_calendar['type'] = daily_calendar['type'].fillna('None')

holiday_types = daily_calendar['type'].unique().tolist()
holiday_type_map = {k: i for i, k in enumerate(holiday_types)}
daily_calendar['holiday_type_encoded'] = daily_calendar['type'].map(holiday_type_map)

daily_calendar['is_holiday'] = (daily_calendar['name'] != 'None').astype(int)

daily_calendar['prev_holiday'] = daily_calendar['is_holiday'].shift(1, fill_value=0)
daily_calendar['next_holiday'] = daily_calendar['is_holiday'].shift(-1, fill_value=0)
daily_calendar['is_long_weekend'] = ((daily_calendar['is_holiday'] == 1) |
                                     (daily_calendar['prev_holiday'] == 1) |
                                     (daily_calendar['next_holiday'] == 1)).astype(int)
daily_calendar.drop(columns=['prev_holiday', 'next_holiday'], inplace=True)

daily_calendar['day_of_week'] = daily_calendar['date'].dt.dayofweek
daily_calendar['month'] = daily_calendar['date'].dt.month

hourly_calendar = pd.DataFrame({
    "timestamp": pd.date_range(start=min_date, end=max_date + pd.Timedelta(days=1) - pd.Timedelta(hours=1), freq='H')
})
hourly_calendar['date_only'] = hourly_calendar['timestamp'].dt.date
hourly_calendar = hourly_calendar.merge(
    daily_calendar.drop(columns=['date']),
    on='date_only',
    how='left'
)
hourly_calendar.drop(columns=['date_only'], inplace=True)
print(hourly_calendar.head())


            timestamp  name  type  holiday_type_encoded  is_holiday  \
0 2021-01-01 00:00:00  None  None                   0.0         0.0   
1 2021-01-01 01:00:00  None  None                   0.0         0.0   
2 2021-01-01 02:00:00  None  None                   0.0         0.0   
3 2021-01-01 03:00:00  None  None                   0.0         0.0   
4 2021-01-01 04:00:00  None  None                   0.0         0.0   

   is_long_weekend  day_of_week  month  
0              0.0          4.0    1.0  
1              0.0          4.0    1.0  
2              0.0          4.0    1.0  
3              0.0          4.0    1.0  
4              0.0          4.0    1.0  


In [8]:
def fetch_weather(city="Delhi"):
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city},IN&limit=1&appid={WEATHER_API_KEY}"
    geo_response = requests.get(geo_url)
    if geo_response.status_code != 200:
        print(f"Error fetching coordinates: {geo_response.status_code}")
        return None
    geo_data = geo_response.json()
    if not geo_data:
        print(f"City '{city}' not found.")
        return None

    lat, lon = geo_data[0]['lat'], geo_data[0]['lon']

    weather_url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&appid={WEATHER_API_KEY}"
    weather_response = requests.get(weather_url)
    if weather_response.status_code != 200:
        print(f"Error fetching weather: {weather_response.status_code}")
        return None

    weather_data = weather_response.json()

    # Current + next 5 forecasts
    current = weather_data["list"][0]
    forecast = weather_data["list"][1:6]

    result = {
        "city": city.title(),
        "coordinates": {"lat": lat, "lon": lon},
        "current": {
            "datetime": current["dt_txt"],
            "temperature": current["main"]["temp"],
            "feels_like": current["main"]["feels_like"],
            "humidity": current["main"]["humidity"],
            "condition": current["weather"][0]["description"],
            "wind_speed": current["wind"]["speed"]
        },
        "forecast": [
            {
                "datetime": f["dt_txt"],
                "temperature": f["main"]["temp"],
                "feels_like": f["main"]["feels_like"],
                "humidity": f["main"]["humidity"],
                "condition": f["weather"][0]["description"],
                "wind_speed": f["wind"]["speed"]
            }
            for f in forecast
        ]
    }

    weather_df = pd.DataFrame(result["forecast"])
    weather_df["city"] = result["city"]
    weather_df["lat"] = lat
    weather_df["lon"] = lon

    return weather_df

weather_df = fetch_weather("Delhi")

if weather_df is not None:
    print("Weather dataset created successfully!")
    print(weather_df.head())
else:
    print("No weather data fetched.")


Weather dataset created successfully!
              datetime  temperature  feels_like  humidity         condition  \
0  2025-10-07 00:00:00        21.67       22.37        95     moderate rain   
1  2025-10-07 03:00:00        22.53       23.03        84        light rain   
2  2025-10-07 06:00:00        27.46       28.60        59  scattered clouds   
3  2025-10-07 09:00:00        30.63       31.33        46         clear sky   
4  2025-10-07 12:00:00        30.64       30.59        41        few clouds   

   wind_speed   city        lat        lon  
0        4.10  Delhi  28.651718  77.221939  
1        4.15  Delhi  28.651718  77.221939  
2        2.99  Delhi  28.651718  77.221939  
3        2.32  Delhi  28.651718  77.221939  
4        2.15  Delhi  28.651718  77.221939  


In [12]:
el = electricity.copy()
el.columns

Index(['Unnamed: 0', 'timestamp', 'Power demand', 'temp', 'dwpt', 'rhum',
       'wdir', 'wspd', 'pres', 'year', 'month', 'day', 'hour', 'minute',
       'moving_avg_3'],
      dtype='object')

In [14]:
print(hourly_calendar.columns)
print(hourly_calendar.head())

Index(['timestamp', 'name', 'type', 'holiday_type_encoded', 'is_holiday',
       'is_long_weekend', 'day_of_week', 'month'],
      dtype='object')
            timestamp  name  type  holiday_type_encoded  is_holiday  \
0 2021-01-01 00:00:00  None  None                   0.0         0.0   
1 2021-01-01 01:00:00  None  None                   0.0         0.0   
2 2021-01-01 02:00:00  None  None                   0.0         0.0   
3 2021-01-01 03:00:00  None  None                   0.0         0.0   
4 2021-01-01 04:00:00  None  None                   0.0         0.0   

   is_long_weekend  day_of_week  month  
0              0.0          4.0    1.0  
1              0.0          4.0    1.0  
2              0.0          4.0    1.0  
3              0.0          4.0    1.0  
4              0.0          4.0    1.0  


In [9]:
# Standardize column names: lowercase, replace spaces and special chars with underscores
new_cols = []
for c in electricity.columns:
    c2 = str(c).strip().lower().replace(' ', '_').replace('-', '_')
    new_cols.append(c2)

electricity.columns = new_cols

# Drop unnamed index-like column if present
if 'unnamed:_0' in electricity.columns:
    electricity = electricity.drop(columns=['unnamed:_0'])

# Rename specific known columns to concise names
rename_map = {
    'datetime': 'timestamp',
    'power_demand': 'demand_mw',
    'temp': 'temp_c',
    'dwpt': 'dewpoint_c',
    'rhum': 'rel_humidity_pct',
    'wdir': 'wind_dir_deg',
    'wspd': 'wind_speed_ms',
    'pres': 'pressure_hpa',
    'moving_avg_3': 'demand_ma3'
}

for k in list(rename_map.keys()):
    if k in electricity.columns:
        pass

electricity = electricity.rename(columns=rename_map)

# Parse timestamp and sort
if 'timestamp' in electricity.columns:
    electricity['timestamp'] = pd.to_datetime(electricity['timestamp'])
    electricity = electricity.sort_values('timestamp')

# Basic time features if not already present
for part, fmt in [('year', '%Y'), ('month', '%m'), ('day', '%d'), ('hour', '%H'), ('minute', '%M')]:
    if part not in electricity.columns and 'timestamp' in electricity.columns:
        electricity[part] = electricity['timestamp'].dt.__getattribute__(part)


print(electricity.shape)

(393440, 14)


In [18]:
print(hourly_calendar.columns)

Index(['timestamp', 'name', 'type', 'holiday_type_encoded', 'is_holiday',
       'is_long_weekend', 'day_of_week', 'month'],
      dtype='object')


In [10]:
# Ensure datetime columns exist and match
if 'timestamp' not in hourly_calendar.columns and 'date' in hourly_calendar.columns:
    hourly_calendar['timestamp'] = pd.to_datetime(hourly_calendar['date'])
else:
    hourly_calendar['timestamp'] = pd.to_datetime(hourly_calendar['timestamp'])

if 'timestamp' not in electricity.columns and 'datetime' in electricity.columns:
    electricity['timestamp'] = pd.to_datetime(electricity['datetime'])
else:
    electricity['timestamp'] = pd.to_datetime(electricity['timestamp'])

# Ensure holiday type column exists in calendar_hourly
if 'encoder__holiday_type_restricted' not in hourly_calendar.columns:
   hourly_calendar['encoder__holiday_type_restricted'] = 0

# Merge electricity with calendar_hourly
electricity = electricity.merge(
    hourly_calendar[["timestamp", "is_holiday", "encoder__holiday_type_restricted", "is_long_weekend"]],
    on="timestamp",
    how="left"
)

# Rename for convenience
electricity.rename(columns={"encoder__holiday_type_restricted": "holiday_type_restricted"}, inplace=True)

# Fill NaNs
electricity["is_holiday"] = electricity["is_holiday"].fillna(0).astype(int)
electricity["is_long_weekend"] = electricity["is_long_weekend"].fillna(0).astype(int)

# Additional features
electricity["quarter"] = electricity["timestamp"].dt.quarter
electricity["hour_of_day"] = electricity["timestamp"].dt.hour
electricity["day_of_week"] = electricity["timestamp"].dt.dayofweek
electricity["is_weekend"] = electricity["day_of_week"].isin([5, 6]).astype(int)

print(electricity.head())


            timestamp  demand_mw  temp_c  dewpoint_c  rel_humidity_pct  \
0 2021-01-01 00:30:00    2014.00     8.0         6.9              93.0   
1 2021-01-01 00:35:00    2005.63     8.0         6.9              93.0   
2 2021-01-01 00:40:00    1977.60     8.0         6.9              93.0   
3 2021-01-01 00:45:00    1976.44     8.0         6.9              93.0   
4 2021-01-01 00:50:00    1954.37     8.0         6.9              93.0   

   wind_dir_deg  wind_speed_ms  pressure_hpa  year  month  ...  hour  minute  \
0           0.0            0.0        1017.0  2021      1  ...     0      30   
1           0.0            0.0        1017.0  2021      1  ...     0      35   
2           0.0            0.0        1017.0  2021      1  ...     0      40   
3           0.0            0.0        1017.0  2021      1  ...     0      45   
4           0.0            0.0        1017.0  2021      1  ...     0      50   

    demand_ma3  is_holiday  holiday_type_restricted  is_long_weekend  quar

In [11]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd

# -------------------------------
# Step 0: Rename columns to match pipeline
# -------------------------------
electricity.rename(columns={
    "Power demand": "demand_mw",
    "temp": "temp_c",
    "dwpt": "dewpoint_c",
    "rhum": "rel_humidity_pct",
    "wdir": "wind_dir_deg",
    "wspd": "wind_speed_ms",
    "pres": "pressure_hpa",
    "datetime": "timestamp"
}, inplace=True)

# -------------------------------
# Step 1: Create lag & rolling features
# -------------------------------
# Sort by timestamp
electricity = electricity.sort_values("timestamp")

# Lag features
lags = [1, 2, 3]
for lag in lags:
    electricity[f"demand_mw_lag_{lag}"] = electricity["demand_mw"].shift(lag)

# Rolling features
windows = [3, 6, 12]
for window in windows:
    electricity[f"demand_mw_roll_mean_{window}"] = electricity["demand_mw"].shift(1).rolling(window).mean()
    electricity[f"demand_mw_roll_std_{window}"] = electricity["demand_mw"].shift(1).rolling(window).std()

# Fill NaNs
electricity.fillna(0, inplace=True)

# -------------------------------
# Step 2: Define features
# -------------------------------
numeric_features = [
    "demand_mw", "temp_c", "dewpoint_c", "rel_humidity_pct",
    "wind_dir_deg", "wind_speed_ms", "pressure_hpa",
    # lag & rolling features
    "demand_mw_lag_1", "demand_mw_lag_2", "demand_mw_lag_3",
    "demand_mw_roll_mean_3", "demand_mw_roll_mean_6", "demand_mw_roll_mean_12",
    "demand_mw_roll_std_3", "demand_mw_roll_std_6", "demand_mw_roll_std_12"
]

categorical_features = ["quarter", "hour_of_day", "day_of_week", "month"]
binary_features = ["is_weekend", "is_holiday", "is_long_weekend"]

# -------------------------------
# Step 3: Pipelines & ColumnTransformer
# -------------------------------
numeric_transformer = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

binary_transformer = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_features)
    ]
)

# -------------------------------
# Step 4: Fit & transform
# -------------------------------
X_preprocessed = preprocessor.fit_transform(electricity)

# Column names
cat_columns = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)
all_columns = numeric_features + list(cat_columns) + binary_features

electricity_cleaned = pd.DataFrame(X_preprocessed, columns=all_columns)
electricity_cleaned.index = electricity["timestamp"]

print("Shape with lag & rolling features:", electricity_cleaned.shape)
print(electricity_cleaned.head())


Shape with lag & rolling features: (393440, 66)
                     demand_mw    temp_c  dewpoint_c  rel_humidity_pct  \
timestamp                                                                
2021-01-01 00:30:00  -1.496946 -2.196053   -1.279313          1.189677   
2021-01-01 00:35:00  -1.503382 -2.196053   -1.279313          1.189677   
2021-01-01 00:40:00  -1.524936 -2.196053   -1.279313          1.189677   
2021-01-01 00:45:00  -1.525828 -2.196053   -1.279313          1.189677   
2021-01-01 00:50:00  -1.542799 -2.196053   -1.279313          1.189677   

                     wind_dir_deg  wind_speed_ms  pressure_hpa  \
timestamp                                                        
2021-01-01 00:30:00     -1.398298      -1.386675      1.168709   
2021-01-01 00:35:00     -1.398298      -1.386675      1.168709   
2021-01-01 00:40:00     -1.398298      -1.386675      1.168709   
2021-01-01 00:45:00     -1.398298      -1.386675      1.168709   
2021-01-01 00:50:00     -1.398298    

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

numeric_features = [
    "demand_mw", "temp_c", "dewpoint_c", "rel_humidity_pct",
    "wind_dir_deg", "wind_speed_ms", "pressure_hpa",
    "demand_mw_lag_1", "demand_mw_lag_2", "demand_mw_lag_3",
    "demand_mw_roll_mean_3", "demand_mw_roll_mean_6", "demand_mw_roll_mean_12",
    "demand_mw_roll_std_3", "demand_mw_roll_std_6", "demand_mw_roll_std_12"
]

categorical_features = ["quarter", "hour_of_day", "day_of_week", "month"]
binary_features = ["is_weekend", "is_holiday", "is_long_weekend"]

target = "demand_mw"

# Preprocessing
X = electricity[numeric_features + categorical_features + binary_features].copy()
y = electricity[target].copy()

# ColumnTransformer
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

binary_transformer = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('bin', binary_transformer, binary_features)
])

X_preprocessed = preprocessor.fit_transform(X)

# Get final column names
cat_columns = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
all_columns = numeric_features + list(cat_columns) + binary_features
X_preprocessed = pd.DataFrame(X_preprocessed.toarray() if hasattr(X_preprocessed, "toarray") else X_preprocessed,
                              columns=all_columns)
X_preprocessed.index = electricity.index

# Train / Val / Test Split (chronological)
n = len(X_preprocessed)
train_end = int(n*0.7)
val_end = int(n*0.85)

X_train = X_preprocessed.iloc[:train_end]
y_train = y.iloc[:train_end]
X_val = X_preprocessed.iloc[train_end:val_end]
y_val = y.iloc[train_end:val_end]
X_test = X_preprocessed.iloc[val_end:]
y_test = y.iloc[val_end:]

# LightGBM Model
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_val, label=y_val)

params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": 42
}

callbacks = [lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=100)]

lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val], valid_names=["train","val"],
                      num_boost_round=2000, callbacks=callbacks)

y_pred_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
print("LGB RMSE:", mean_squared_error(y_test, y_pred_lgb, squared=False))
print("LGB MAE:", mean_absolute_error(y_test, y_pred_lgb))

# LSTM Model
timesteps = 24

def create_lstm_dataset(X, y, timesteps):
    Xs, ys = [], []
    for i in range(timesteps, len(X)):
        Xs.append(X.iloc[i-timesteps:i].values)
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

X_train_lstm, y_train_lstm = create_lstm_dataset(X_train, y_train, timesteps)
X_val_lstm, y_val_lstm = create_lstm_dataset(X_val, y_val, timesteps)
X_test_lstm, y_test_lstm = create_lstm_dataset(X_test, y_test, timesteps)

# Build LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, X_train_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train_lstm, y_train_lstm, validation_data=(X_val_lstm, y_val_lstm), epochs=50, batch_size=64,
          callbacks=[early_stop])

y_pred_lstm = model.predict(X_test_lstm).flatten()
print("LSTM RMSE:", mean_squared_error(y_test_lstm, y_pred_lstm, squared=False))
print("LSTM MAE:", mean_absolute_error(y_test_lstm, y_pred_lstm))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3723
[LightGBM] [Info] Number of data points in the train set: 275408, number of used features: 64
[LightGBM] [Info] Start training from score 3837.594041
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 12.852	val's rmse: 38.4414
Early stopping, best iteration is:
[127]	train's rmse: 9.76839	val's rmse: 37.1728
LGB RMSE: 105.70918658656115
LGB MAE: 27.437884130931145
Epoch 1/50
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 15139899.0000 - mae: 3686.6499 - val_loss: 11174588.0000 - val_mae: 3132.8049
Epoch 2/50
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 11500718.0000 - mae: 3155.8650 - val_loss: 8147912.5000 - val_mae: 2605.3376
Epoch 3/50
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss

In [15]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define cross-validation
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

X = X_preprocessed.copy()  
y = electricity[target].copy() 

# Store metrics
lgb_rmse_list, lgb_mae_list = [], []
lstm_rmse_list, lstm_mae_list = [], []

timesteps = 24  # for LSTM

def create_lstm_dataset(X, y, timesteps):
    Xs, ys = [], []
    for i in range(timesteps, len(X)):
        Xs.append(X.iloc[i-timesteps:i].values)
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

split_num = 1
for train_index, test_index in tscv.split(X):
    print(f"=== Split {split_num} ===")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # LightGBM
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_val = lgb.Dataset(X_test, label=y_test)
    params = {
        "objective": "regression",
        "metric": "rmse",
        "learning_rate": 0.05,
        "num_leaves": 31,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "seed": 42
    }
    lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_val], num_boost_round=1000,
                          callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=500)])
    
    y_pred_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    lgb_rmse_list.append(mean_squared_error(y_test, y_pred_lgb, squared=False))
    lgb_mae_list.append(mean_absolute_error(y_test, y_pred_lgb))

    # LSTM
    X_train_lstm, y_train_lstm = create_lstm_dataset(X_train, y_train, timesteps)
    X_test_lstm, y_test_lstm = create_lstm_dataset(X_test, y_test, timesteps)

    model = Sequential()
    model.add(LSTM(64, input_shape=(timesteps, X_train_lstm.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train_lstm, y_train_lstm, validation_data=(X_test_lstm, y_test_lstm),
              epochs=30, batch_size=64, callbacks=[early_stop], verbose=0)
    
    y_pred_lstm = model.predict(X_test_lstm).flatten()
    lstm_rmse_list.append(mean_squared_error(y_test_lstm, y_pred_lstm, squared=False))
    lstm_mae_list.append(mean_absolute_error(y_test_lstm, y_pred_lstm))
    
    print(f"LGB RMSE: {lgb_rmse_list[-1]:.2f}, MAE: {lgb_mae_list[-1]:.2f}")
    print(f"LSTM RMSE: {lstm_rmse_list[-1]:.2f}, MAE: {lstm_mae_list[-1]:.2f}\n")
    
    split_num += 1

print("=== Cross-validation Summary ===")
print(f"LGB mean RMSE: {np.mean(lgb_rmse_list):.2f} ± {np.std(lgb_rmse_list):.2f}")
print(f"LGB mean MAE: {np.mean(lgb_mae_list):.2f} ± {np.std(lgb_mae_list):.2f}")
print(f"LSTM mean RMSE: {np.mean(lstm_rmse_list):.2f} ± {np.std(lstm_rmse_list):.2f}")
print(f"LSTM mean MAE: {np.mean(lstm_mae_list):.2f} ± {np.std(lstm_mae_list):.2f}")


=== Split 1 ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3555
[LightGBM] [Info] Number of data points in the train set: 65575, number of used features: 59
[LightGBM] [Info] Start training from score 3535.431650
Training until validation scores don't improve for 50 rounds
[500]	valid_0's rmse: 9.16549
[1000]	valid_0's rmse: 9.03278
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 9.03278
2049/2049 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
LGB RMSE: 9.03, MAE: 5.28
LSTM RMSE: 1910.14, MAE: 1626.26

=== Split 2 ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 

In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def regression_metrics(y_true, y_pred, name):
    #Align lengths safely
    n = min(len(y_true), len(y_pred))
    y_true = np.array(y_true[:n])
    y_pred = np.array(y_pred[:n])

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
    wape = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

    abs_error = np.abs(y_true - y_pred)
    p50 = np.percentile(abs_error, 50)
    p90 = np.percentile(abs_error, 90)
    p95 = np.percentile(abs_error, 95)

    print(f"\n=== {name} Model Metrics ===")
    print(f"MSE  : {mse:.2f}")
    print(f"RMSE : {rmse:.2f}")
    print(f"MAE  : {mae:.2f}")
    print(f"MAPE : {mape:.2f}%")
    print(f"WAPE : {wape:.2f}%")
    print(f"P50 Error : {p50:.2f}")
    print(f"P90 Error : {p90:.2f}")
    print(f"P95 Error : {p95:.2f}")

regression_metrics(y_test, y_pred_lgb, "LightGBM")
regression_metrics(y_test, y_pred_lstm, "LSTM")



=== LightGBM Model Metrics ===
MSE  : 8766.61
RMSE : 93.63
MAE  : 23.85
MAPE : 0.37%
WAPE : 0.50%
P50 Error : 5.78
P90 Error : 24.76
P95 Error : 66.09

=== LSTM Model Metrics ===
MSE  : 338869.16
RMSE : 582.12
MAE  : 419.74
MAPE : 9.71%
WAPE : 8.81%
P50 Error : 334.56
P90 Error : 832.96
P95 Error : 1069.98


In [21]:
import pickle

with open("lgb_model.pkl", "wb") as f:
    pickle.dump(lgb_model, f)

print("LightGBM model saved as 'lgb_model.pkl'")

LightGBM model saved as 'lgb_model.pkl'
